# Memory

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [22]:
store = {}  # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [23]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you']

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'), 
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

In [25]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [26]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 농장에서 나무를 세 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 농장에서 나무를 세 그루 키우고 있다니 재미있네요! 어떤 종류의 나무를 키우고 있나요? 혹시 나무에 대해 더 이야기해주실 수 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 42, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BOgytqNXvma7Bc6qcNS2sjYrOrKZz', 'finish_reason': 'stop', 'logprobs': None} id='run-8ceea44d-4e36-44d8-ba9b-725221e6b2b7-0' usage_metadata={'input_tokens': 42, 'output_tokens': 48, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [27]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 재미있네요! 어떤 종류의 나무를 키우고 있나요? 혹시 나무에 대해 더 이야기해주실 수 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 42, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BOgytqNXvma7Bc6qcNS2sjYrOrKZz', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ceea44d-4e36-44d8-ba9b-725221e6b2b7-0', usage_metadata={'input_tokens': 42, 'output_tokens': 48, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_deta

In [29]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 사과 나무를 다섯 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='다람쥐가 사과 나무를 다섯 그루 키우고 있다니, 정말 멋지네요! 사과 나무는 다람쥐에게 맛있는 간식을 제공할 뿐만 아니라, 그늘도 만들어줄 수 있겠네요. 다람쥐가 사과를 어떻게 관리하는지, 혹은 사과를 수확한 후 어떻게 사용하는지에 대한 이야기가 있을까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 222, 'total_tokens': 315, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BOgzOQ1IWmw49D2TxFBuPQXDEWZkr', 'finish_reason': 'stop', 'logprobs': None} id='run-d0ac099e-4089-441a-9101-2df6d0a883f5-0' usage_metadata={'input_tokens': 222, 'output_tokens': 93, 'total_tokens': 315, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [30]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'rabbit': [HumanMessage(content='토끼는 농장에서 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 농장에서 나무를 세 그루 키우고 있다니 재미있네요! 어떤 종류의 나무를 키우고 있나요? 혹시 나무에 대해 더 이야기해주실 수 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 42, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BOgytqNXvma7Bc6qcNS2sjYrOrKZz', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ceea44d-4e36-44d8-ba9b-725221e6b2b7-0', usage_metadata={'input_tokens': 42, 'output_tokens': 48, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_deta

In [31]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼와 다람쥐는 합해서 몇 그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 나무를 세 그루 키우고, 다람쥐가 다섯 그루 키우고 있으니, 합쳐서 총 8그루의 나무를 키우고 있습니다. (3 + 5 = 8) 정말 멋진 농장이네요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 346, 'total_tokens': 411, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BOh1vfhT8F92VWdQ89IuldYMYP2F1', 'finish_reason': 'stop', 'logprobs': None} id='run-93093077-bd0e-449d-a98d-4cc473e33533-0' usage_metadata={'input_tokens': 346, 'output_tokens': 65, 'total_tokens': 411, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
